In [1]:
pip install pandas sqlalchemy requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import os
import sqlite3

def download_file(url, filename):
    """
    Downloads a file from a given URL and saves it with the specified filename.

    Args:
        url (str): The URL of the file to download.
        filename (str): The name to save the file as.
    """
    # Check if the file already exists to avoid re-downloading
    if os.path.exists(filename):
        print(f"'{filename}' already exists. Skipping download.")
        return

    print(f"Downloading {filename} from {url}...")
    try:
        # Use requests to get the file content. stream=True allows for large files.
        with requests.get(url, stream=True) as r:
            r.raise_for_status()  # Raise an exception for bad status codes (4xx or 5xx)
            with open(filename, 'wb') as f:
                # Write the file content in chunks to save memory
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
        print(f"Successfully downloaded and saved '{filename}'.")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading '{filename}': {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

def list_tables(db_file):
    """
    Connects to a SQLite database and prints a list of all tables.

    Args:
        db_file (str): The path to the SQLite database file.
    """
    print("-" * 30)
    print(f"Listing tables in '{db_file}':")
    try:
        # Connect to the database
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()

        # Execute the query to find all tables in the database
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        if tables:
            for table in tables:
                print(f"- {table[0]}")
        else:
            print("No tables found in this database.")

        # Close the connection
        conn.close()
    except sqlite3.Error as e:
        print(f"SQLite error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("-" * 30)

# Define the URLs for the databases. These are direct links to the raw files on GitHub.
CHINOOK_URL = "https://raw.githubusercontent.com/lerocha/chinook-database/master/ChinookDatabase/DataSources/Chinook_Sqlite.sqlite"
# Updated URL for the Northwind database to resolve the 404 error
NORTHWIND_URL = "https://github.com/jpwhite3/northwind-SQLite3/raw/main/dist/northwind.db"

# Define the desired filenames
chinook_filename = "chinook.db"
northwind_filename = "northwind.db"

# --- Main script execution ---

# Download the databases
download_file(CHINOOK_URL, chinook_filename)
download_file(NORTHWIND_URL, northwind_filename)

print("\nAll download operations complete. Checking your current directory for the database files.")

# List the tables in each database
list_tables(chinook_filename)
list_tables(northwind_filename)

'chinook.db' already exists. Skipping download.
Successfully downloaded and saved 'northwind.db'.

All download operations complete. Checking your current directory for the database files.
------------------------------
Listing tables in 'chinook.db':
- Album
- Artist
- Customer
- Employee
- Genre
- Invoice
- InvoiceLine
- MediaType
- Playlist
- PlaylistTrack
- Track
------------------------------
------------------------------
Listing tables in 'northwind.db':
- Categories
- sqlite_sequence
- CustomerCustomerDemo
- CustomerDemographics
- Customers
- Employees
- EmployeeTerritories
- Order Details
- Orders
- Products
- Regions
- Shippers
- Suppliers
- Territories
------------------------------


In [5]:
from sqlalchemy import create_engine
import pandas as pd

# Create an SQLAlchemy engine for the Chinook database
chinook_engine = create_engine("sqlite:///chinook.db", echo=False)

# A connection can be explicitly opened and closed
chinook_conn = chinook_engine.connect()

# Or, as a best practice, use a 'with' statement for automatic resource management
with chinook_engine.connect() as conn:
    print("Connection to Chinook database established successfully.")
    # All database operations would happen here
    pass

chinook_conn.close() # Close the explicit connection

Connection to Chinook database established successfully.


In [7]:
from sqlalchemy import create_engine

# Create an SQLAlchemy engine for the Northwind database
northwind_engine = create_engine("sqlite:///northwind.db", echo=False)

with northwind_engine.connect() as conn:
    print("Connection to Northwind database established successfully.")
    # All database operations would happen here
    pass


Connection to Northwind database established successfully.


In [9]:
from sqlalchemy import inspect

def list_columns_in_database(engine, db_name):
    """
    Connects to a database and prints a list of all tables and their columns.

    Args:
        engine (sqlalchemy.engine.base.Engine): The SQLAlchemy engine for the database.
        db_name (str): The name of the database (for printing purposes).
    """
    print("-" * 30)
    print(f"Listing tables and columns in '{db_name}':")
    try:
        inspector = inspect(engine)
        table_names = inspector.get_table_names()

        if table_names:
            for table_name in table_names:
                print(f"\nTable: {table_name}")
                columns = inspector.get_columns(table_name)
                for column in columns:
                    print(f"  - {column['name']} ({column['type']})")
        else:
            print("No tables found in this database.")

    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    finally:
        print("-" * 30)

In [11]:
# List tables and columns for Chinook database
list_columns_in_database(chinook_engine, "Chinook")

------------------------------
Listing tables and columns in 'Chinook':

Table: Album
  - AlbumId (INTEGER)
  - Title (NVARCHAR(160))
  - ArtistId (INTEGER)

Table: Artist
  - ArtistId (INTEGER)
  - Name (NVARCHAR(120))

Table: Customer
  - CustomerId (INTEGER)
  - FirstName (NVARCHAR(40))
  - LastName (NVARCHAR(20))
  - Company (NVARCHAR(80))
  - Address (NVARCHAR(70))
  - City (NVARCHAR(40))
  - State (NVARCHAR(40))
  - Country (NVARCHAR(40))
  - PostalCode (NVARCHAR(10))
  - Phone (NVARCHAR(24))
  - Fax (NVARCHAR(24))
  - Email (NVARCHAR(60))
  - SupportRepId (INTEGER)

Table: Employee
  - EmployeeId (INTEGER)
  - LastName (NVARCHAR(20))
  - FirstName (NVARCHAR(20))
  - Title (NVARCHAR(30))
  - ReportsTo (INTEGER)
  - BirthDate (DATETIME)
  - HireDate (DATETIME)
  - Address (NVARCHAR(70))
  - City (NVARCHAR(40))
  - State (NVARCHAR(40))
  - Country (NVARCHAR(40))
  - PostalCode (NVARCHAR(10))
  - Phone (NVARCHAR(24))
  - Fax (NVARCHAR(24))
  - Email (NVARCHAR(60))

Table: Genre
  - 

In [13]:
# List tables and columns for Northwind database
list_columns_in_database(northwind_engine, "Northwind")

------------------------------
Listing tables and columns in 'Northwind':

Table: Categories
  - CategoryID (INTEGER)
  - CategoryName (TEXT)
  - Description (TEXT)
  - Picture (BLOB)

Table: CustomerCustomerDemo
  - CustomerID (TEXT)
  - CustomerTypeID (TEXT)

Table: CustomerDemographics
  - CustomerTypeID (TEXT)
  - CustomerDesc (TEXT)

Table: Customers
  - CustomerID (TEXT)
  - CompanyName (TEXT)
  - ContactName (TEXT)
  - ContactTitle (TEXT)
  - Address (TEXT)
  - City (TEXT)
  - Region (TEXT)
  - PostalCode (TEXT)
  - Country (TEXT)
  - Phone (TEXT)
  - Fax (TEXT)

Table: EmployeeTerritories
  - EmployeeID (INTEGER)
  - TerritoryID (TEXT)

Table: Employees
  - EmployeeID (INTEGER)
  - LastName (TEXT)
  - FirstName (TEXT)
  - Title (TEXT)
  - TitleOfCourtesy (TEXT)
  - BirthDate (DATE)
  - HireDate (DATE)
  - Address (TEXT)
  - City (TEXT)
  - Region (TEXT)
  - PostalCode (TEXT)
  - Country (TEXT)
  - HomePhone (TEXT)
  - Extension (TEXT)
  - Photo (BLOB)
  - Notes (TEXT)
  - Repor

In [15]:
# Load the entire artists table from Chinook
chinook_artists_df = pd.read_sql_table('Artist', con=chinook_engine, )
print("\nChinook Artists DataFrame loaded successfully.")
chinook_artists_df.head()


Chinook Artists DataFrame loaded successfully.


,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [17]:
# Load the entire Employees table from Northwind
northwind_employees_df = pd.read_sql_table('Employees', con=northwind_engine)
print("Northwind Employees DataFrame loaded successfully.")
northwind_employees_df.head()

Northwind Employees DataFrame loaded successfully.


,EmployeeID,LastName,FirstName,Title,TitleOfCourtesy,BirthDate,HireDate,Address,City,Region,PostalCode,Country,HomePhone,Extension,Photo,Notes,ReportsTo,PhotoPath
0,1,Davolio,Nancy,Sales Representative,Ms.,1968-12-08,2012-05-01,507 - 20th Ave. E.Apt. 2A,Seattle,North America,98122,USA,(206) 555-9857,5467,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,Education includes a BA in psychology from Col...,2.0,http://accweb/emmployees/davolio.bmp
1,2,Fuller,Andrew,"Vice President, Sales",Dr.,1972-02-19,2012-08-14,908 W. Capital Way,Tacoma,North America,98401,USA,(206) 555-9482,3457,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,Andrew received his BTS commercial in 1974 and...,NaN,http://accweb/emmployees/fuller.bmp
2,3,Leverling,Janet,Sales Representative,Ms.,1983-08-30,2012-04-01,722 Moss Bay Blvd.,Kirkland,North America,98033,USA,(206) 555-3412,3355,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,Janet has a BS degree in chemistry from Boston...,2.0,http://accweb/emmployees/leverling.bmp
3,4,Peacock,Margaret,Sales Representative,Mrs.,1957-09-19,2013-05-03,4110 Old Redmond Rd.,Redmond,North America,98052,USA,(206) 555-8122,5176,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,Margaret holds a BA in English literature from...,2.0,http://accweb/emmployees/peacock.bmp
4,5,Buchanan,Steven,Sales Manager,Mr.,1975-03-04,2013-10-17,14 Garrett Hill,London,British Isles,SW1 8JR,UK,(71) 555-4848,3453,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x02\x00...,Steven Buchanan graduated from St. Andrews Uni...,2.0,http://accweb/emmployees/buchanan.bmp


In [19]:
artists_albums_query = """
SELECT
    T1.Name AS ArtistName,
    T2.Title AS AlbumTitle
FROM Artist AS T1
JOIN Album AS T2
    ON T1.ArtistId = T2.ArtistId
ORDER BY
    ArtistName
LIMIT 10;
"""
artists_albums_df = pd.read_sql_query(artists_albums_query, con=chinook_engine)
print("\nDataFrame of Chinook artists and their albums loaded successfully.")
artists_albums_df


DataFrame of Chinook artists and their albums loaded successfully.


,ArtistName,AlbumTitle
0,AC/DC,For Those About To Rock We Salute You
1,AC/DC,Let There Be Rock
2,Aaron Copland & London Symphony Orchestra,"A Copland Celebration, Vol. I"
3,Aaron Goldberg,Worlds
4,Academy of St. Martin in the Fields & Sir Nevi...,The World of Classical Favourites
5,Academy of St. Martin in the Fields Chamber En...,Sir Neville Marriner: A Celebration
6,"Academy of St. Martin in the Fields, John Birc...","Fauré: Requiem, Ravel: Pavane & Others"
7,"Academy of St. Martin in the Fields, Sir Nevil...",Bach: Orchestral Suites Nos. 1 - 4
8,Accept,Balls to the Wall
9,Accept,Restless and Wild
